## Path Association ##

In [122]:
import cv2
import os
import shutil
import pandas as pd
import json
import numpy as np
import cv2
import math
import random

#参考https://blog.csdn.net/zimiao552147572/article/details/105985322
#https://blog.csdn.net/weixin_41387192/article/details/110197467最近邻搜索
# https://blog.csdn.net/qq_34446614/article/details/117401217#:~:text=KNN%E7%9A%84%E5%B7%A5%E4%BD%9C%E5%8E%9F%E7%90%86%EF%BC%9A%E7%BB%99 KNN
#线段拟合https://blog.csdn.net/weixin_44789544/article/details/104646452
#曲线拟合 https://blog.csdn.net/qq_34802028/article/details/119351263

#首先将属于统一直线的线段进行搜索算法，加入到同一个集合之中
def NNDA(lines):



#将多个线段的拟合成曲线
def draw_curve(img_save_dir):
    output = cv2.fitLine(InputArray  points, distType, param, reps, aeps)




In [123]:
import sys
sys.path.append(r'../PeterPan/')
import eval_yolo_results2
import YoloTools as yt
import glob, os

# --------------可视化线段关联验证流程, 5个步骤依次运行-------------------
img_path = r'../../Highway_2024_0510/output/0628_data2/deci_7_7/pic/0.png' 
patch_dir =    r'../../Highway_2024_0510/output/0628_data2/deci_7_7/patches4'
yolo_predict = r'../../Highway_2024_0510/output/0628_data2/deci_7_7/yolo/labels'   # 每次预测完会存入新的文件夹，需修改“exp”对应的数字

img_flist = glob.glob('../../Highway_2024_0510/output/0628_data2/deci_7_7/pic5/*.png')
# 1.img to patch, 待验证图像划分patches保存
if not os.path.exists(patch_dir):
    os.makedirs(patch_dir)
for img_path in img_flist:
    pathc_num, img_w,img_h= eval_yolo_results2.img2patch_pad(img_path, patch_dir, patch_size=64, padding=16)


In [124]:
# 2. yolo predict, 用训练好的yolo模型预测上一步骤中的patches
# 命令：python detect.py --weights ./runs/train/exp7/weights/best.pt --source ../datasets/traffic/images/val --imgsz 192 --iou-thres 0.5 --conf-thres 0.5 --save-txt
# 命令：python ../../yolo5/detect.py --weights ./runs/train/exp7/weights/best.pt --source ../../Highway_2024_0510/output/0628_data2/deci_7_7/patches --project '../../Highway_2024_0510/output/0628_data2/' --name 'deci_7_7/yolo/' --imgsz 192 --iou-thres 0.5 --conf-thres 0.5 --save-txt
# ! python ../../Vehicle_Trace_DL/yolov5/detect.py --weights ../../Vehicle_Trace_DL/yolov5/runs/train/exp7/weights/best.pt --source ../../Highway_2024_0510/output/0628_data2/deci_7_7/patches4 --project '../../Highway_2024_0510/output/0628_data2/' --name 'deci_7_7/yolo/' --imgsz 192 --iou-thres 0.5 --conf-thres 0.5 --save-txt

In [125]:
yolo_predict = r'../../Highway_2024_0510/output/0628_data2/deci_7_7/yolo12/labels'   # 每次预测完会存入新的文件夹，需修改“exp”对应的数字
img_save_dir = r"../../Highway_2024_0510/output/0628_data2/deci_7_7/predict_vis_result18"
# 3. draw predicted lines to patch, 将预测结果的txt文件合并到json文件中，json文件存放在结果文件夹中
if not os.path.exists(img_save_dir):
    os.makedirs(img_save_dir)
img_flist.sort()
print(len(img_flist))
for img_path in img_flist:
    img_name = img_path.split("/")[-1].replace(".png","")
    txt_save_path = os.path.join(img_save_dir,img_name+'.txt')
    yt.txt2json(img_name,yolo_predict,txt_save_path,img_w,img_h ,padding=16 , patch_num = 75 , patch_size=64)

# txt2line(yolo_predict,patch_dir)
# 4.将patch图片进行拼接
for img_path in img_flist:
    img_name = img_path.split("/")[-1].replace(".png","")
    img_save_path = os.path.join(img_save_dir,img_name+'.png')
    yt.patch2img_pad(patch_dir,img_name, img_save_path,img_w,img_h, padding=16 , patch_num=pathc_num)

# # 5. json文件中的线段，画在拼接好的完整图片上
yt.draw_line(img_save_dir)

#395 

10
52
59
62
55
58
64
50
53
49
45
[114, 96]
[193, 105]
[321, 98]
[385, 109]
[83, 129]
[129, 166]
[360, 127]
[321, 137]
[384, 153]
[147, 191]
[193, 236]
[260, 204]
[296, 193]
[321, 222]
[383, 212]
[425, 192]
[385, 196]
[149, 257]
[131, 257]
[192, 291]
[214, 257]
[319, 262]
[359, 257]
[446, 263]
[384, 279]
[65, 339]
[109, 320]
[129, 337]
[278, 319]
[319, 364]
[430, 321]
[65, 425]
[184, 387]
[181, 384]
[131, 391]
[193, 394]
[295, 385]
[302, 383]
[336, 385]
[321, 398]
[105, 449]
[128, 476]
[138, 449]
[266, 450]
[319, 489]
[399, 447]
[385, 455]
[128, 519]
[176, 511]
[209, 512]
[281, 513]
[332, 512]
[190, 115]
[168, 96]
[321, 109]
[384, 110]
[175, 129]
[318, 174]
[306, 127]
[351, 129]
[410, 129]
[383, 158]
[87, 193]
[128, 232]
[140, 192]
[256, 233]
[303, 192]
[385, 244]
[426, 192]
[159, 255]
[189, 288]
[193, 301]
[227, 256]
[200, 257]
[193, 281]
[400, 256]
[126, 342]
[172, 320]
[150, 319]
[132, 321]
[191, 369]
[261, 328]
[381, 328]
[384, 355]
[447, 336]
[425, 319]
[105, 384]
[132, 387]
[246, 